In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from tqdm import tqdm

from xgboost import XGBClassifier
from xgboost.core import XGBoostError

import csv

import cupy as cp

import gc

In [2]:
df = pd.read_csv("../../story_dataset.csv")
df

,prompt_id,prompt,story,hidden_state_file,len_generated_story
0,1,Once upon a time there was a dragon,Once upon a time there was a dragon. It was bi...,./hidden_states/prompt_1.npz,273
1,1,Once upon a time there was a dragon,Once upon a time there was a dragon. He loved ...,./hidden_states/prompt_1.npz,246
2,1,Once upon a time there was a dragon,Once upon a time there was a dragon named Sam....,./hidden_states/prompt_1.npz,397
3,1,Once upon a time there was a dragon,Once upon a time there was a dragon. He was a ...,./hidden_states/prompt_1.npz,294
4,1,Once upon a time there was a dragon,Once upon a time there was a dragon. The drago...,./hidden_states/prompt_1.npz,296
...,...,...,...,...,...
9995,10,Once upon a time there was a poor boy,Once upon a time there was a poor boy named Ti...,./hidden_states/prompt_10.npz,315
9996,10,Once upon a time there was a poor boy,Once upon a time there was a poor boy named Ti...,./hidden_states/prompt_10.npz,270
9997,10,Once upon a time there was a poor boy,Once upon a time there was a poor boy named Ti...,./hidden_states/prompt_10.npz,206
9998,10,Once upon a time there was a poor boy,Once upon a time there was a poor boy named Ti...,./hidden_states/prompt_10.npz,375


In [3]:
# hidden_states_by_layer = {}
# curr_labels = []
# NUM_PROMPTS = 10
# # NUM_PROMPTS = 2
# context_level = 49

# #Load hidden states to generate first token
# for prompt_id in range(1, NUM_PROMPTS + 1):
#     with np.load(f'../../hidden_states/prompt_{prompt_id}.npz') as loaded_data:
#         for i in tqdm(range(1000)):
#             curr_hidden_states = loaded_data[f"arr_{i}"]
#             # print(curr_hidden_states.shape)
#             if(context_level > curr_hidden_states.shape[0]):
#                 continue

#             curr_hidden_states = curr_hidden_states[context_level]
#             curr_labels.append(df.iloc[i * prompt_id].prompt_id)
            
#             #By layer
#             for layer in range(1):
#                 curr_layer_hidden_states = curr_hidden_states[layer][0].astype('float32') #FAISS expects data in type float32 instead of float64 - saves memory too!
#                 # print(curr_layer_hidden_states.shape)
                
#                 if(f"layer_{layer}" in hidden_states_by_layer):
#                     hidden_states_by_layer[f"layer_{layer}"].append(curr_layer_hidden_states)
#                 else:
#                     hidden_states_by_layer[f"layer_{layer}"] = [curr_layer_hidden_states]

In [4]:
hidden_states_by_cl = {}
curr_labels = {}
NUM_PROMPTS = 10
layer = 0

In [5]:
max_story_len = max(df[:NUM_PROMPTS*1000]["len_generated_story"])
max_story_len

522

In [6]:
df[df["len_generated_story"] >= max_story_len]

,prompt_id,prompt,story,hidden_state_file,len_generated_story
9851,10,Once upon a time there was a poor boy,Once upon a time there was a poor boy named Ti...,./hidden_states/prompt_10.npz,522


In [7]:
min_story_len = min(df["len_generated_story"])
min_story_len

37

In [8]:
context_levels = [0] #get hs used to generate second token
context_levels.extend([i for i in range(23, max_story_len, 25)]) #actual context level is context_level + 2 (gather hs to generate the context_level + 2 token)

In [9]:
# #results using hs to generate first token already captured in other notebook
# #get hs to generate second token, 25th token, 50th token, etc.

# #Load hidden states to generate first token
# for prompt_id in range(1, NUM_PROMPTS + 1):
#     with np.load(f'../../hidden_states/prompt_{prompt_id}.npz') as loaded_data:
#         for i in tqdm(range(1000)):
#             for context_level in context_levels:
#                 curr_hidden_states = loaded_data[f"arr_{i}"]
#                 # print(curr_hidden_states.shape)
#                 if(context_level > curr_hidden_states.shape[0] - 1):
#                     continue
                
#                 curr_hidden_states = curr_hidden_states[context_level][layer][0].astype('float32')
                
#                 if(f"cl_{context_level + 2}" in hidden_states_by_cl):
#                     hidden_states_by_cl[f"cl_{context_level + 2}"].append(curr_hidden_states)
#                     curr_labels[f"cl_{context_level + 2}"].append(df.iloc[i * prompt_id].prompt_id)
#                 else:
#                     hidden_states_by_cl[f"cl_{context_level + 2}"] = [curr_hidden_states]
#                     curr_labels[f"cl_{context_level + 2}"] = [df.iloc[i * prompt_id].prompt_id]

In [10]:
hidden_states_by_cl = {f"cl_{cl+2}": [] for cl in context_levels}
curr_labels = {f"cl_{cl+2}": [] for cl in context_levels}

for prompt_id in range(1, NUM_PROMPTS + 1):
    with np.load(f'../../hidden_states/prompt_{prompt_id}.npz') as loaded_data:

        for i in tqdm(range(1000)):
            hs = loaded_data[f"arr_{i}"]  # load once

            for context_level in context_levels:
                if context_level >= hs.shape[0]:
                    continue

                key = f"cl_{context_level + 2}"

                hidden_states_by_cl[key].append(
                    hs[context_level][layer][0].astype("float32")
                )
                curr_labels[key].append(prompt_id - 1)


100%|███████████████████████████████████████████████████████████████████████████████| 1000/1000 [00:23<00:00, 43.26it/s]


In [11]:
def safe_split(total, train_ratio=0.8):
    train = int(train_ratio * total)   # floor
    test  = total - train              # leftover
    return train, test

def build_dataset(curr_context_level_hs, curr_labels, train_ratio=0.8):
    """
    Groups samples by prompt_id, splits safely into train/test,
    and returns X_train, y_train, X_test, y_test.
    """

    unique_ids = sorted(set(curr_labels))

    X_train_list = []
    X_test_list = []
    y_train_list = []
    y_test_list = []

    for pid in unique_ids:

        # get samples for this prompt id
        mask = (curr_labels == pid)
        X_pid = curr_context_level_hs[mask]
        y_pid = curr_labels[mask]

        total = len(X_pid)
        train_n, test_n = safe_split(total, train_ratio)

        # split
        X_train_list.append(X_pid[:train_n])
        y_train_list.append(y_pid[:train_n])

        X_test_list.append(X_pid[train_n:])
        y_test_list.append(y_pid[train_n:])

    # concatenate all prompt-id blocks
    X_train = np.concatenate(X_train_list, axis=0)
    y_train = np.concatenate(y_train_list, axis=0)
    X_test  = np.concatenate(X_test_list, axis=0)
    y_test  = np.concatenate(y_test_list, axis=0)

    # return cp.array(X_train), cp.array(y_train), cp.array(X_test), cp.array(y_test)
    return X_train, y_train, X_test, y_test
    

In [12]:
np.array(hidden_states_by_cl[f"cl_2"]).shape

(10000, 1, 512)

In [13]:
print(np.array(curr_labels[f"cl_2"]).shape)

(10000,)


In [17]:
# with open("results.csv", "w+", newline='') as csvfile:
#     csv_writer = csv.writer(csvfile, delimiter=',')
#     header = ['Layer', 'Context_Level', 'Train_Accuracy', 'Test_Accuracy']
#     prompt_headers = []
#     for i in range(1, 11):
#         prompt_headers.extend([f"Prompt_{i}_Train_Accuracy", f"Prompt_{i}_Test_Accuracy"])

#     header.extend(prompt_headers)
#     csv_writer.writerow(header)

for context_level in context_levels:
    cl_hs_array = hidden_states_by_cl[f"cl_{context_level + 2}"]
    curr_label_set = curr_labels[f"cl_{context_level + 2}"]
    # print(cl_hs_array[0].shape)
    # print(cl_hs_array[-1].shape)
    #1x512

    print(np.array(cl_hs_array).shape)
    for hs in range(len(cl_hs_array)):
        cl_hs_array[hs] = cl_hs_array[hs].flatten()

    curr_context_level_hs = np.array(cl_hs_array)
    print(curr_context_level_hs.shape)

    curr_label_set = np.array(curr_label_set)

    unique_ids = sorted(set(curr_label_set))

    if(len(unique_ids) < 10): break
    
    X_train, y_train, X_test, y_test = build_dataset(curr_context_level_hs, curr_label_set)

    #split into train and test and see how many samples of each class are in the test set (might explain 0.0 acc performance in test set).
    # print(sorted(set(y_train)))
    # print(sorted(set(y_test)))
    print(pd.Series(y_train).value_counts())
    print(pd.Series(y_test).value_counts())

    print("Train Data Shape: ", X_train.shape)
    print("Test Data Shape: ", X_test.shape)
    
    X_train = cp.array(X_train)
    y_train = cp.array(y_train)
    X_test = cp.array(X_test)
    y_test = cp.array(y_test)

    classifier = XGBClassifier(seed = 42, objective = 'multi:softmax', eval_metric = "merror", num_class = len(unique_ids), tree_method='hist', device='cuda')
    classifier.fit(X_train, y_train)
    preds_train = classifier.predict(X_train)
    preds = classifier.predict(X_test)

    train_accuracy = np.mean(cp.array(preds_train) == y_train)
    accuracy = np.mean(cp.array(preds) == y_test)

    print(f"Train Accuracy: {train_accuracy}")
    print(f"Test Accuracy: {accuracy}")

    prompt_accs = []

    for i in range(10):
        mask = y_test == i
        prompt_test = y_test[mask]
        prompt_preds = cp.array(preds)[mask]
        prompt_test_acc = np.mean(prompt_preds == prompt_test)

        mask_train = y_train == i
        prompt_train = y_train[mask_train]
        prompt_preds_train = cp.array(preds_train)[mask_train]
        prompt_train_acc = np.mean(prompt_preds_train == prompt_train)

        print(f"Prompt {i + 1} Train Accuracy: {prompt_train_acc}")
        print(f"Prompt {i + 1} Test Accuracy: {prompt_test_acc}")

        prompt_accs.append(prompt_train_acc)
        prompt_accs.append(prompt_test_acc)

    
    # with open("results.csv", "a+", newline='') as csvfile:
    #     csv_writer = csv.writer(csvfile, delimiter=',')
    #     values = [layer, context_level + 2, train_accuracy, accuracy]
    #     values.extend(prompt_accs)
    #     csv_writer.writerow(values)
            


    del classifier, X_train, y_train, X_test, y_test, preds

    # XGBoost cleanup
    try:
        booster = classifier.get_booster()
        del booster
    except:
        pass
    
    # CuPy cleanup
    cp.get_default_memory_pool().free_all_blocks()
    cp.get_default_pinned_memory_pool().free_all_blocks()
    
    gc.collect()

(10000, 512)
(10000, 512)
0    800
1    800
2    800
3    800
4    800
5    800
6    800
7    800
8    800
9    800
Name: count, dtype: int64
0    200
1    200
2    200
3    200
4    200
5    200
6    200
7    200
8    200
9    200
Name: count, dtype: int64
Train Data Shape:  (8000, 512)
Test Data Shape:  (2000, 512)
Train Accuracy: 0.271625
Test Accuracy: 0.272
Prompt 1 Train Accuracy: 0.00375
Prompt 1 Test Accuracy: 0.0
Prompt 2 Train Accuracy: 0.0
Prompt 2 Test Accuracy: 0.0
Prompt 3 Train Accuracy: 0.59375
Prompt 3 Test Accuracy: 0.655
Prompt 4 Train Accuracy: 0.0075
Prompt 4 Test Accuracy: 0.0
Prompt 5 Train Accuracy: 0.97875
Prompt 5 Test Accuracy: 0.995
Prompt 6 Train Accuracy: 0.14
Prompt 6 Test Accuracy: 0.1
Prompt 7 Train Accuracy: 0.88875
Prompt 7 Test Accuracy: 0.86
Prompt 8 Train Accuracy: 0.0
Prompt 8 Test Accuracy: 0.0
Prompt 9 Train Accuracy: 0.10375
Prompt 9 Test Accuracy: 0.11
Prompt 10 Train Accuracy: 0.0
Prompt 10 Test Accuracy: 0.0
(10000, 512)
(10000, 512)
0    80

In [15]:
# for key in curr_labels.keys():
#     print(pd.Series(curr_labels[key]).value_counts())

#     #split into train and test and see how many samples of each class are in the test set (might explain 0.0 acc performance in test set).
#     X_train, y_train, X_test, y_test = build_dataset(curr_context_level_hs, curr_label_set)